## Importing Libraries

In [0]:
#Implementation of Incremental Mathematical Stream Regression
from sklearn import metrics  #importing datasets and metrics from the sklearn library
import pandas as pd
import numpy as np  

## Loading Dataset

In [0]:
testdataframe=pd.read_csv("H:/AOBD/boston.csv",header=None)  #load dataset from laptop
testdataarray=testdataframe.values  #convert the dataframe to array
X_test=testdataarray[400:506,:-1]  #separate the last 106 datapoints from data for test dataset
y_test=testdataarray[400:506,-1]  #separate the last 106 datapoints output variable for test dataset

## Python Data Generator

In [0]:
def myGeneratorfun(wsize=100):
    print("In myGeneratorfun:")
    counter=0  #counter to stop stream data after all data is used
    for chunk in pd.read_csv('H:/AOBD/boston.csv', header=None, chunksize=wsize):  #for loop to separate out data in chunks
        print("In for loop, counter vlaue is", counter)
        chunk_array=chunk.values  #Each chunk of data is equal to window size and convert it to array
        counter=counter+1  #increment counter by 1 know that one more chunk of data is used
        yield (chunk_array[:,:-1], chunk_array[:,-1])  #This yield statement returns chunk of data 
        if counter >= 4:  #the for loop will continue to send chunks of data till the if statement is not satisfied
            break  #after all the data is used as chunks

## ASR Implementation

![asr.JPG](attachment:asr.JPG)

![asr1.JPG](attachment:asr1.JPG)

In [0]:
myGenerator = myGeneratorfun(100)  #Call the data Generator with window size=100

Beta_k = np.zeros(X_test.shape[1])  #initialize Beta_k with zeros of p*1 shape
Beta_k_dash = np.zeros(X_test.shape[1])  #initialize Beta_k_dash with zeros of p*1 shape

#alpha = 0.5  #smoothing factor for ASR=0.5, equal weightage for New data and Previous Data
t=5 #initializing t as half life 

alpha= math.exp(-((math.log(2))/t)) #calculating alpha
for (x,y) in myGenerator:  #to call for windows one-by-one window
        
    try: 
        XTXi=inv(np.dot(x.T,x))  #XTX for current window if invertible
    except LinAlgError:
        XTXi=pinv(np.dot(x.T,x))  #If non-invertible
        
    XTy = np.dot(x.T,y)  #XTy for current window
    Beta_k_dash = np.dot(XTXi,(XTy))  #Calculate parameters for current window
    Beta_k = np.dot((1-alpha),Beta_k_dash)+np.dot(alpha,Beta_k)  # ASR Formula to calculate parameters using pre-synopis

    #print("Betas:", Beta)
    #print("Betas Shape:", Beta.shape)   

    predictions=np.dot(Beta_k,X_test.T) # Calculate Predictions on test data
    print(predictions.shape)
    
    # Print Mean Absolute Error And Mean Squared Error
    print("MAE:", metrics.mean_absolute_error(y_true=y_test,y_pred=predictions))
    print("MSE:", metrics.mean_squared_error(y_true=y_test,y_pred=predictions))

In myGeneratorfun:
In for loop, counter vlaue is 0
(106,)
MAE: 26.162161684337455
MSE: 1111.6177456133041
In for loop, counter vlaue is 1
(106,)
MAE: 4.294020071584694
MSE: 30.171732802407305
In for loop, counter vlaue is 2
(106,)
MAE: 4.464642658408654
MSE: 30.705804122398295
In for loop, counter vlaue is 3
(106,)
MAE: 3.2698325654573788
MSE: 18.28457339686843
